In [1]:
from datetime import datetime
import os
import pandas as pd
import numpy as np
from scipy.stats import norm, zscore
from arch import arch_model
from copulas.multivariate import GaussianMultivariate
import configparser
from alpaca.trading.client import TradingClient
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from alpaca.data.enums import DataFeed
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

# --- Setup: API keys and clients
def initialize_clients():
    config = configparser.ConfigParser()
    config.read('local.ini')
    API_KEY = config.get('Alpaca-Paper-Trading', 'API_KEY')
    API_SECRET = config.get('Alpaca-Paper-Trading', 'API_SECRET')
    trading_client = TradingClient(API_KEY, API_SECRET, paper=True)
    data_client = StockHistoricalDataClient(API_KEY, API_SECRET)
    return trading_client, data_client

# --- Define stock list and date
def get_stock_params():
    today_str = datetime.today().strftime("%Y-%m-%d")
    return {                               ### Representation of the Technology sector ###
        "META": ("2023-01-01", today_str), # F
        "AAPL": ("2023-01-01", today_str), # A
        "AMZN": ("2023-01-01", today_str), # A
        "NFLX": ("2023-01-01", today_str), # N
        "GOOG": ("2023-01-01", today_str), # G
        "MSFT": ("2023-01-01", today_str), # + Microsoft
        "NVDA": ("2023-01-01", today_str), # + Nvidia
    }

# --- Fetch full data and save CSVs (overwrite)
def fetch_all_data(data_client, stock_params):
    for symbol, (start, end) in stock_params.items():
        bars_req = StockBarsRequest(
            symbol_or_symbols=[symbol],
            timeframe=TimeFrame.Day,
            start=start,
            end=end,
            feed=DataFeed.IEX
        )
        bars_result = data_client.get_stock_bars(bars_req)
        print(f"{symbol}: Data fetched from {start} to {end} with shape {bars_result.df.shape}")
        bars_result.df.to_csv(f"case_data/{symbol}.csv", index=True)

# --- Compute clean log returns
def compute_log_returns(stock_params):
    log_returns = {}
    for symbol in stock_params.keys():
        df = pd.read_csv(f"case_data/{symbol}.csv", index_col=1, parse_dates=True)
        df[f"{symbol}_log"] = np.log(df['close'] / df['close'].shift(1))
        log_returns[f"{symbol}_log"] = df[f"{symbol}_log"]
    data = pd.DataFrame(log_returns).dropna()
    data_clean = data[(np.abs(zscore(data)) < 6).all(axis=1)]
    return data_clean

# --- Fit GARCH models and return standardized residuals
def fit_garch(data_clean):
    resids = pd.DataFrame()
    for col in data_clean.columns:
        series = 100 * data_clean[col].dropna().iloc[-252:]
        model = arch_model(series, vol='Garch', p=1, q=1, dist='normal')
        fitted = model.fit(disp='off')
        resids[col] = fitted.resid / fitted.conditional_volatility
    return resids

# --- Generate signals for the latest date using copula likelihoods
def generate_signals(resids, stock_params):
    copula = GaussianMultivariate()
    copula.fit(resids)

    resids_uniform = pd.DataFrame(norm.cdf(resids), columns=resids.columns, index=resids.index)
    likelihoods = copula.probability_density(resids_uniform)

    lower, upper = np.percentile(likelihoods, 0), np.percentile(likelihoods, 30)
    extreme_days = (likelihoods >= lower) & (likelihoods < upper)

    latest_date = resids.index.max()
    idx = resids.index.get_loc(latest_date)
    today_likelihood = likelihoods[idx]

    print(f"\n--- META Likelihood Report ---")
    print(f"Latest Date:       {latest_date.date()}")
    print(f"Likelihood:        {today_likelihood:.6f}")
    print(f"Threshold (0-30%): [{lower:.6f}, {upper:.6f})")
    print(f"In extreme region: {extreme_days[idx]}")
    print("------------------------------\n")

    signals_dict = {}
    for asset in stock_params.keys():
        col = f"{asset}_log"
        if latest_date not in resids.index:
            continue
        if extreme_days[idx]:
            signal = 'buy' if resids.at[latest_date, col] < 0 else 'sell'
        else:
            signal = 'hold'
        signals_dict[asset] = signal

    return signals_dict


# --- Trade logic only for META
def trade_meta_only(trading_client, signals_dict, account):
    meta_signal = signals_dict.get("META")
    cash = float(account.cash)
    positions = {p.symbol: float(p.qty) for p in trading_client.get_all_positions()}

    if meta_signal == 'sell' and "META" in positions and positions["META"] > 0:
        qty = int(positions["META"])
        order = MarketOrderRequest("META", qty, OrderSide.SELL, TimeInForce.DAY)
        trading_client.submit_order(order)
        print(f"Selling {qty} shares of META")

    elif meta_signal == 'buy' and cash > 0:
        df = pd.read_csv("case_data/META.csv", index_col=1, parse_dates=True)
        price = df['close'].iloc[-1]
        qty = int(cash // price)
        if qty > 0:
            order = MarketOrderRequest("META", qty, OrderSide.BUY, TimeInForce.DAY)
            trading_client.submit_order(order)
            print(f"Buying {qty} shares of META at price {price:.2f}")
        else:
            print("Not enough cash to buy META shares")
    else:
        print(f"Signal: {meta_signal}, Cash: {cash:.2f}")

# --- Main trading function
def run_trading():
    trading_client, data_client = initialize_clients()
    print(f"Today's date: {datetime.today().strftime('%Y-%m-%d')}")

    account = trading_client.get_account()
    print(f"Account status: {account.status}")
    print(f"Cash balance:   ${account.cash}")

    stock_params = get_stock_params()
    fetch_all_data(data_client, stock_params)  # Fetch full data fresh every run
    data_clean = compute_log_returns(stock_params)
    resids = fit_garch(data_clean)
    signals_dict = generate_signals(resids, stock_params)
    trade_meta_only(trading_client, signals_dict, account)




In [3]:
run_trading() # Run once


Today's date: 2025-07-29
Account status: AccountStatus.ACTIVE
Cash balance:   $100000
META: Data fetched from 2023-01-01 to 2025-07-29 with shape (643, 7)
AAPL: Data fetched from 2023-01-01 to 2025-07-29 with shape (643, 7)
AMZN: Data fetched from 2023-01-01 to 2025-07-29 with shape (643, 7)
NFLX: Data fetched from 2023-01-01 to 2025-07-29 with shape (643, 7)
GOOG: Data fetched from 2023-01-01 to 2025-07-29 with shape (643, 7)
MSFT: Data fetched from 2023-01-01 to 2025-07-29 with shape (643, 7)
NVDA: Data fetched from 2023-01-01 to 2025-07-29 with shape (643, 7)

--- META Likelihood Report ---
Latest Date:       2025-07-28
Likelihood:        0.004807
Threshold (0-30%): [0.001602, 0.003476)
In extreme region: False
------------------------------

Signal: hold, Cash: 100000.00


In [ ]:
import time
if __name__ == "__main__":
    for _ in range(10): # Run 10 days
        try:
            run_trading()
        except Exception as e:
            print(f"Error: {e}")
        time.sleep(86400)  # Sleep 24 hours

Today's date: 2025-07-18
Account status: AccountStatus.ACTIVE
Cash balance:   $100000
META: Data fetched from 2023-01-01 to 2025-07-18 with shape (636, 7)
AAPL: Data fetched from 2023-01-01 to 2025-07-18 with shape (636, 7)
AMZN: Data fetched from 2023-01-01 to 2025-07-18 with shape (636, 7)
NFLX: Data fetched from 2023-01-01 to 2025-07-18 with shape (636, 7)
GOOG: Data fetched from 2023-01-01 to 2025-07-18 with shape (636, 7)
MSFT: Data fetched from 2023-01-01 to 2025-07-18 with shape (636, 7)
NVDA: Data fetched from 2023-01-01 to 2025-07-18 with shape (636, 7)
Generating signals for latest available date: 2025-07-17
No trade action taken for META today. Signal: hold, Cash: 100000.00
